In [1]:
# # Uncomment and run this if you do not have the following packages
# using Pkg
# # LEAP model
# Pkg.add(url="https://github.com/tyhlee/LEAP.jl")
# # # changing the input for simulation model
# Pkg.add("Setfield")
# # save output
# Pkg.add("JLD")
# Pkg.add("JLD2")
# Pkg.add("CodecZlib")
# # Optional: parallel computation
# # start your jupyter notebook with X number of threads you want to use
# Pkg.add("Distributed")

In [2]:
# import packages
using LEAP, Setfield, JLD, JLD2
using Distributed

In [3]:
# check the number of threads for parallel computation
Threads.nthreads()

6

In [4]:
# createa a folder for results
results_dir = "../results";
mkdir(results_dir)

"../results"

In [5]:
# three scenarios
# set max age to 111, province to British Columbia (BC), starting year to 2001, and run it for 18 cycles/years
base_model = LEAP.set_up(111,"BC",2001,18,"full","M3");
flat_model = base_model;
@set! flat_model.antibioticExposure.parameters[:fixyear] = 2001;
mid_model =  base_model;
@set! mid_model.antibioticExposure.parameters[:fixyear] = "mid";

In [6]:
# ensure that the model works
test_model = base_model;
# set the simulation to 0.1% for a test run
@set! test_model.n = 0.001; 
# args: model, rng seed, run until everyone dies, verbose
test_run = LEAP.process(test_model,0,false,true);
# ~5s on my machine

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018

 Simulation finished. Check your simulation object for results. ───────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
       Tot / % measured:           5.39s /  94.0%           2.67GiB /  97.5%    

 Section              ncalls     time    %tot     avg     alloc    %tot      avg
 ───────────────────────────────────────────────────────────────────────────────
 calendar year 2001        1    4.33s   85.5%   4.33s   2.21GiB   84.9%  2.21GiB
 calendar year 2002        1    215ms    4.2%   215ms   45.7MiB    1.7%  45.7MiB
 calendar year 2006        1   48.1ms    0.9%  48.1ms   31.3MiB    1.2%  31.3MiB
 calendar year 2004        1   42.2ms    0.8%  42.2ms   28.6MiB    1.1%  28.6MiB
 calendar year 2013        1   39.0

In [7]:
# This is a test run for parallel computation
# test
test_results_dir = "../test_results";
mkdir(test_results_dir)

n = 0.001
N = 6
@set! base_model.n = n;
@set! flat_model.n = n;
@set! mid_model.n = n;

Threads.@threads for i = 1:N
    tmp_dir = string(test_results_dir,"/",i,"")
    mkdir(tmp_dir)
    JLD2.jldsave(string(tmp_dir,"/","base.jld"),true; outcome = LEAP.process(base_model,i,false,false))
    JLD2.jldsave(string(tmp_dir,"/","flat.jld"),true; outcome = LEAP.process(flat_model,i,false,false))
    JLD2.jldsave(string(tmp_dir,"/","mid.jld"),true; outcome = LEAP.process(mid_model,i,false,false))
end

In [8]:
n = 0.1
N = 1/n * 100
@set! base_model.n = n;
@set! flat_model.n = n;
@set! mid_model.n = n;

Threads.@threads for i = 1:N
    i = Int(i)
    tmp_dir = string(results_dir,"/",i,"")
    mkdir(tmp_dir)
    JLD2.jldsave(string(tmp_dir,"/","base.jld"),true; outcome = LEAP.process(base_model,i,false,false))
    JLD2.jldsave(string(tmp_dir,"/","flat.jld"),true; outcome = LEAP.process(flat_model,i,false,false))
    JLD2.jldsave(string(tmp_dir,"/","mid.jld"),true; outcome = LEAP.process(mid_model,i,false,false))
end